In [1]:
# -----------------------------------------------------------------------
# IMPORTS
# -----------------------------------------------------------------------
# Tratamiento de datos
# -------------------------------------------------
import pandas as pd
import numpy as np
import re

# Visualización
# -------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# -------------------------------------------------
from scipy.stats import shapiro, kstest

# Configuración
# -------------------------------------------------
pd.set_option('display.max_columns', None)      # Para visualizar todas las columnas de los DataFrames.
pd.set_option('display.max_colwidth', None)     # Para ver todo el contenido de las columnas.

# Gestión de los warnings
# -------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# GENDER QUOTES

In [2]:
df = pd.read_csv("files/friends_quotes.csv")

In [3]:
df.head(5)

,author,episode_number,episode_title,quote,quote_order,season
0,Monica,1.0,Monica Gets A Roommate,There's nothing to tell! He's just some guy I work with!,0.0,1.0
1,Joey,1.0,Monica Gets A Roommate,"C'mon, you're going out with the guy! There's gotta be something wrong with him!",1.0,1.0
2,Chandler,1.0,Monica Gets A Roommate,"All right Joey, be nice. So does he have a hump? A hump and a hairpiece?",2.0,1.0
3,Phoebe,1.0,Monica Gets A Roommate,"Wait, does he eat chalk?",3.0,1.0
4,Phoebe,1.0,Monica Gets A Roommate,"Just, 'cause, I don't want her to go through what I went through with Carl- oh!",4.0,1.0


In [4]:
df['author'].unique().tolist()

['Monica',
 'Joey',
 'Chandler',
 'Phoebe',
 'All',
 'Ross',
 'Rachel',
 'Waitress',
 'Phoebe, Ross, Chandler, and Joey',
 'Paul',
 'Priest on TV',
 'Frannie',
 'Ross and Rachel',
 'Customer',
 'Marsha',
 'Carol',
 'Chandler and Joey',
 'Mrs. Geller',
 'Mr. Geller',
 'Susan',
 'Barry',
 'Robbie',
 'Bernice',
 'Dr. Oberman',
 'Paula',
 'Alan',
 'The Guys',
 'Lizzie',
 'Guys',
 'Leslie',
 'Kiki',
 'Joanne',
 'Receptionist',
 'Pizza Guy',
 'Monica and Phoebe',
 'Both',
 'Kid',
 'Angela',
 'Janice',
 'Woman',
 'Bob',
 "All you want is a dingle,\r\n What you envy's a schwang,\r\n A thing through which you can tinkle,\r\n Or play with, or simply let hang..",
 'Aurora',
 'Director',
 'Jill',
 'Monica, Joey, and Phoebe',
 'Mr. Heckles',
 'Paolo',
 'Shelley',
 'Aunt Lillian',
 'Nurse',
 'Andrea',
 'Lowell',
 'Terry',
 'Guy',
 'Girl',
 'Bladder Control Problem\r\n Stop Wife Beating\r\n Hemorrhoids?\r\n Winner of 3 Tony Awards...',
 "He's finally happy with that and walks away.",
 "I made a man w

In [5]:
df.columns

Index(['author', 'episode_number', 'episode_title', 'quote', 'quote_order',
       'season'],
      dtype='object')

In [6]:
df["author_clean"] = (
    df["author"]
    .astype(str)
    .str.strip()
    .str.title())

In [7]:
corrections = {
    "Rache": "Rachel",
    "Monic": "Monica",
    "Chandle": "Chandler",
    "Phoeb": "Phoebe",
    "Ros": "Ross"}

df["author_clean"] = df["author_clean"].replace(corrections)


In [8]:
df.columns

Index(['author', 'episode_number', 'episode_title', 'quote', 'quote_order',
       'season', 'author_clean'],
      dtype='object')

In [9]:
df.columns

Index(['author', 'episode_number', 'episode_title', 'quote', 'quote_order',
       'season', 'author_clean'],
      dtype='object')

In [10]:
df['author_clean'].unique().tolist()

['Monica',
 'Joey',
 'Chandler',
 'Phoebe',
 'All',
 'Ross',
 'Rachel',
 'Waitress',
 'Phoebe, Ross, Chandler, And Joey',
 'Paul',
 'Priest On Tv',
 'Frannie',
 'Ross And Rachel',
 'Customer',
 'Marsha',
 'Carol',
 'Chandler And Joey',
 'Mrs. Geller',
 'Mr. Geller',
 'Susan',
 'Barry',
 'Robbie',
 'Bernice',
 'Dr. Oberman',
 'Paula',
 'Alan',
 'The Guys',
 'Lizzie',
 'Guys',
 'Leslie',
 'Kiki',
 'Joanne',
 'Receptionist',
 'Pizza Guy',
 'Monica And Phoebe',
 'Both',
 'Kid',
 'Angela',
 'Janice',
 'Woman',
 'Bob',
 "All You Want Is A Dingle,\r\n What You Envy'S A Schwang,\r\n A Thing Through Which You Can Tinkle,\r\n Or Play With, Or Simply Let Hang..",
 'Aurora',
 'Director',
 'Jill',
 'Monica, Joey, And Phoebe',
 'Mr. Heckles',
 'Paolo',
 'Shelley',
 'Aunt Lillian',
 'Nurse',
 'Andrea',
 'Lowell',
 'Terry',
 'Guy',
 'Girl',
 'Bladder Control Problem\r\n Stop Wife Beating\r\n Hemorrhoids?\r\n Winner Of 3 Tony Awards...',
 "He'S Finally Happy With That And Walks Away.",
 "I Made A Man W

In [11]:
gender_map = {
    # Principales
    "Monica": "F",
    "Rachel": "F",
    "Phoebe": "F",
    "Joey": "M",
    "Chandler": "M",
    "Ross": "M",

    # Familia y recurrentes
    "Carol": "F",
    "Susan": "F",
    "Ben": "M",
    "Emma": "F",
    "Jack": "M",
    "Judy": "F",
    "Mr. Geller": "M",
    "Mrs. Geller": "F",
    "Mr. Tribbiani": "M",
    "Mrs. Tribbiani": "F",
    "Grandma Tribbiani": "F",
    "Frank": "M",
    "Frank Jr.": "M",
    "Frank Sr.": "M",
    "Ursula": "F",
    "Amy": "F",

    # Parejas / intereses
    "Barry": "M",
    "Janice": "F",
    "Paolo": "M",
    "Richard": "M",
    "Julie": "F",
    "Bonnie": "F",
    "Emily": "F",
    "Mike": "M",
    "Kathy": "F",
    "Joshua": "M",
    "Pete": "M",
    "Charlie": "F",
    "Mona": "F",
    "Gavin": "M",
    "Tag": "M",
    "Elizabeth": "F",
    "Mark": "M",
    "Paul": "M",
    "Alan": "M",
    "Mindy": "F",
    "Chip": "M",
    "Gary": "M",
    "Roy": "M",
    "Danny": "M",
    "Erin": "F",
    "Janine": "F",
    "Eddie": "M",
    "Joanna": "F",
    "Kate": "F",

    # Amigos / secundarios
    "Aurora": "F",
    "Jill": "F",
    "Angela": "F",
    "Bob": "M",
    "Frannie": "F",
    "Leslie": "F",
    "Kiki": "F",
    "Joanne": "F",
    "Andrea": "F",
    "Lowell": "M",
    "Terry": "M",
    "Max": "M",
    "David": "M",
    "Sandy": "M",
    "Fun Bobby": "M",
    "Roger": "M",
    "Ronni": "F",
    "Lorraine": "F",
    "Kristin": "F",
    "Celia": "F",
    "Steve": "M",
    "Bobby": "M",
    "Danielle": "F",
    "Gerston": "M",
    "Santos": "M",
    "Petrie": "M",
    "Lydia": "F",
    "Melanie": "F",
    "Carl": "M",
    "Paulo": "M",
    "Frankie": "M",
    "Annabelle": "F",
    "Alison": "F",
    "Duncan": "M",
    "Jade": "F",
    "Stephanie": "F",
    "Scott": "M",
    "Ryan": "M",
    "Jeannie": "F",
    "Susie": "F",
    "Erica": "F",
    "Amber": "F",
    "Tilly": "F",
    "Issac": "M",
    "Chloe": "F",
    "Sergei": "M",
    "Mischa": "M",
    "Sophie": "F",
    "Alice": "F",
    "Vince": "M",
    "Jason": "M",
    "Doug": "M",
    "Hoshi": "F",
    "Phil": "M",
    "Stevens": "M",
    "Zoe": "F",
    "Lisa": "F",
    "Mac": "M",
    "Hilda": "F",
    "Melissa": "F",
    "Whitney": "F",
    "Helene": "F",
    "Marjorie": "F",
    "Cynthia": "F",
    "Krista": "F",
    "Kara": "F",
    "Katie": "F",
    "Jen": "F",
    "Kim": "F",
    "Nancy": "F",
    "Kyle": "M",
    "Raymond": "M",
    "Caitlin": "F",
    "Brenda": "F",
    "Stu": "M",
    "Will": "M",
    "Don": "M",
    "Jim": "M",
    "Parker": "M",
    "Ray": "M",
    "Marc": "M",
    "Cliff": "M",
    "Sid": "M",
    "Sally": "F",
    "Hayley": "F",
    "Ken": "M",
    "Claudia": "F",
    "Catherine": "F",
    "Bitsy": "F",
    "Theodore": "M",
    "Molly": "F",
    "Heather": "F",
    "Lennart": "M",
    "Manny": "M",
    "Kori": "F",
    "Chase": "M",
    "Jane": "F",
    "Dirk": "M",
    "Zack": "M",
    "Jeanette": "F",
    "Alexandra": "F",
    "Colleen": "F",
    "Owen": "M",
    "Glenda": "F",
    "Laura": "F",
    "Missy": "F",
    "Donny": "M",
    "Gene": "M",
    "Henrietta": "F",
    "Adrienne": "F",
    "Rita": "F",
    "Mackenzie": "F"}

In [12]:
corrections = {
    "Rache": "Rachel",
    "Monic": "Monica",
    "Chandle": "Chandler",
    "Phoeb": "Phoebe",
    "Ros": "Ross"}

df['author_clean'] = df['author_clean'].replace(corrections)

In [13]:
df.columns

Index(['author', 'episode_number', 'episode_title', 'quote', 'quote_order',
       'season', 'author_clean'],
      dtype='object')

In [14]:
df.columns

Index(['author', 'episode_number', 'episode_title', 'quote', 'quote_order',
       'season', 'author_clean'],
      dtype='object')

In [15]:
df["gender"] = df["author_clean"].map(gender_map)

In [16]:
df[df["gender"].isna()]["author_clean"].value_counts()

author_clean
All                                 330
Rach                                138
Gunther                             130
Mnca                                107
Woman                               100
                                   ... 
Commercial                            1
Monica (Puzzled)                      1
Donny Osmond                          1
Roy, The Male Stripper                1
Phoebe, Ross, Chandler, And Joey      1
Name: count, Length: 789, dtype: int64

In [17]:
df.columns

Index(['author', 'episode_number', 'episode_title', 'quote', 'quote_order',
       'season', 'author_clean', 'gender'],
      dtype='object')

In [18]:
# Número de valores nulos en 'gender'
nulos = df['gender'].isnull().sum()

# Número de valores no nulos en 'gender'
no_nulos = df['gender'].notnull().sum()

print(f"Nulos en gender: {nulos}")
print(f"No nulos en gender: {no_nulos}")

Nulos en gender: 5226
No nulos en gender: 55065


In [19]:
df = df.dropna(subset=['gender'])

In [20]:
# Número de valores nulos en 'gender'
nulos = df['gender'].isnull().sum()

# Número de valores no nulos en 'gender'
no_nulos = df['gender'].notnull().sum()

print(f"Nulos en gender: {nulos}")
print(f"No nulos en gender: {no_nulos}")

Nulos en gender: 0
No nulos en gender: 55065


In [21]:
"Chandler" in df['author'].unique().tolist()

True

In [22]:
df['author_clean'].unique().tolist()

['Monica',
 'Joey',
 'Chandler',
 'Phoebe',
 'Ross',
 'Rachel',
 'Paul',
 'Frannie',
 'Carol',
 'Mrs. Geller',
 'Mr. Geller',
 'Susan',
 'Barry',
 'Alan',
 'Leslie',
 'Kiki',
 'Joanne',
 'Angela',
 'Janice',
 'Bob',
 'Aurora',
 'Jill',
 'Paolo',
 'Andrea',
 'Lowell',
 'Terry',
 'Max',
 'David',
 'Sandy',
 'Fun Bobby',
 'Roger',
 'Mr. Tribbiani',
 'Ronni',
 'Mrs. Tribbiani',
 'Lorraine',
 'Kristin',
 'Celia',
 'Steve',
 'Ursula',
 'Bobby',
 'Mindy',
 'Danielle',
 'Gerston',
 'Santos',
 'Petrie',
 'Lydia',
 'Melanie',
 'Carl',
 'Julie',
 'Paulo',
 'Frankie',
 'Annabelle',
 'Alison',
 'Duncan',
 'Jade',
 'Stephanie',
 'Erica',
 'Susie',
 'Roy',
 'Richard',
 'Eddie',
 'Amber',
 'Tilly',
 'Ben',
 'Frank',
 'Scott',
 'Ryan',
 'Jeannie',
 'Mike',
 'Mark',
 'Issac',
 'Chloe',
 'Sergei',
 'Mischa',
 'Sophie',
 'Alice',
 'Pete',
 'Kate',
 'Joanna',
 'Vince',
 'Jason',
 'Doug',
 'Hoshi',
 'Bonnie',
 'Phil',
 'Stevens',
 'Chip',
 'Kathy',
 'Joshua',
 'Emily',
 'Marjorie',
 'Danny',
 'Cynthia',
 'K

In [23]:
df['author_clean'].isna().any()

np.False_

In [24]:
df.to_csv('files/friends_quotes_gender.csv', index=False)

# GENDER EPISODES V3

In [25]:
df2 = pd.read_csv("files/friends_episodes_v3.csv", encoding="latin-1")

In [26]:
df2.head(5)

,Year_of_prod,Season,Episode Number,Episode_Title,Duration,Summary,Director,Stars,Votes
0,1994,1,1,The One Where Monica Gets a Roommate: The Pilot,22,"Monica and the gang introduce Rachel to the ""real world"" after she leaves her fiancé at the altar.",James Burrows,8.3,7440
1,1994,1,2,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel returns her engagement ring to Barry. Monica becomes stressed when her and Ross's parents come to visit.,James Burrows,8.1,4888
2,1994,1,3,The One with the Thumb,22,Monica becomes irritated when everyone likes her new boyfriend more than she does. Chandler resumes his smoking habit. Phoebe is given $7000 when she finds a thumb in a can of soda.,James Burrows,8.2,4605
3,1994,1,4,The One with George Stephanopoulos,22,"Joey and Chandler take Ross to a hockey game to take his mind off the anniversary of the first time he slept with Carol, while the girls become depressed when they realize they don't have a 'plan'.",James Burrows,8.1,4468
4,1994,1,5,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends his washroom is rat-infested so he can join her at the laundromat. Meanwhile, Joey has Monica pose as his girlfriend, and Chandler struggles to break up with his girlfriend.",Pamela Fryman,8.5,4438


In [27]:
df2['Director'].unique().tolist()

['James Burrows',
 'Pamela Fryman',
 'Arlene Sanford',
 'Peter Bonerz',
 'Paul Lazarus',
 'Alan Myerson',
 'Michael Lembeck',
 'Gail Mancuso',
 'Robby Benson',
 'Kevin Bright',
 'Ellen Gittelsohn',
 'Mary Kay Place',
 'Thomas Schlamme',
 'Steve Zuckerman',
 'Terry Hughes',
 'Sam Simon',
 'Shelley Jensen',
 'Gary Halvorson',
 'David Steinberg',
 'Dana De Vally Piazza',
 'Joe Regalbuto',
 'Andrew Tsao',
 'Todd Holland',
 'David Schwimmer',
 'Ben Weiss',
 'Stephen Prime',
 'Sheldon Epps',
 'Roger Christiansen']

In [28]:
gender_map_dir = {
    'James Burrows': 'M',
    'Pamela Fryman': 'F',
    'Arlene Sanford': 'F',
    'Peter Bonerz': 'M',
    'Paul Lazarus': 'M',
    'Alan Myerson': 'M',
    'Michael Lembeck': 'M',
    'Gail Mancuso': 'F',
    'Robby Benson': 'M',
    'Kevin Bright': 'M',
    'Ellen Gittelsohn': 'F',
    'Mary Kay Place': 'F',
    'Thomas Schlamme': 'M',
    'Steve Zuckerman': 'M',
    'Terry Hughes': 'M',
    'Sam Simon': 'M',
    'Shelley Jensen': 'F',
    'Gary Halvorson': 'M',
    'David Steinberg': 'M',
    'Dana De Vally Piazza': 'F',
    'Joe Regalbuto': 'M',
    'Andrew Tsao': 'M',
    'Todd Holland': 'M',
    'David Schwimmer': 'M',
    'Ben Weiss': 'M',
    'Stephen Prime': 'M',
    'Sheldon Epps': 'M',
    'Roger Christiansen': 'M'}

In [29]:
df2['gender'] = df2['Director'].map(gender_map_dir)

In [30]:
df2[df2['gender'].isna()]['Director'].unique()

array([], dtype=object)

In [31]:
df2.to_csv('files/friends_episodes_v3_gender.csv', index=False)

# GENDER INFO

In [32]:
df3 = pd.read_csv("files/friends_info.csv", encoding="latin-1")

In [33]:
df3.head(5)

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating
0,1,1,The Pilot,James Burrows,David Crane & Marta Kauffman,1994-09-22,21.5,8.3
1,1,2,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,1994-09-29,20.2,8.1
2,1,3,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-10-06,19.5,8.2
3,1,4,The One with George Stephanopoulos,James Burrows,Alexa Junge,1994-10-13,19.7,8.1
4,1,5,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,1994-10-20,18.6,8.5


In [34]:
df3['written_by'].unique().tolist()

['David Crane & Marta Kauffman',
 'Jeffrey Astrof & Mike Sikowitz',
 'Alexa Junge',
 'Jeff Greenstein & Jeff Strauss',
 'Adam Chase & Ira Ungerleider',
 'Marta Kauffman & David Crane',
 'Jeffrey Astrof & Mike Sikowitz & Adam Chase & Ira Ungerleider',
 'Bill Lawrence',
 'Doty Abrams',
 'Story byâ\x80\x8a: David Crane & Marta KauffmanTeleplay byâ\x80\x8a: Jeff Greenstein & Jeff Strauss',
 'Chris Brown',
 'Michael Curtis & Gregory S. Malins',
 'Betsy Borns',
 'Michael Borkow',
 'Ira Ungerleider',
 'Jeffrey Astrof & Mike SikowitzMichael Borkow',
 'Adam Chase',
 'Story byâ\x80\x8a: Alexa JungeTeleplay byâ\x80\x8a: Michael Borkow',
 'Story byâ\x80\x8a: Michael Curtis & Gregory S. MalinsTeleplay byâ\x80\x8a: Adam Chase',
 'Brian Buckner & Sebastian Jones',
 'Brown Mandell',
 'Story byâ\x80\x8a: Ira UngerleiderTeleplay byâ\x80\x8a: Brown Mandell',
 'Wil Calhoun',
 'Shana Goldberg-Meehan & Scott Silveri',
 'Seth Kurland',
 'Gregory S. Malins & Michael Curtis',
 'Ted Cohen & Andrew Reich',
 'Sto

In [35]:
df3['written_by'] = df3['written_by'].replace({
    'Story byâ: Alexa JungeTeleplay byâ: Michael Borkow': 'Alexa Junge,Michael Borkow',
    'Story byâ: Ira UngerleiderTeleplay byâ: Brown Mandell': 'Ira Ungerleider,Brown Mandell'})

In [36]:
# Limpiar problemas de encoding y texto fijo
df3['written_by_clean'] = (
    df3['written_by']
    .str.replace(r'â\\x80\\x8a', ' ', regex=True)
    .str.replace(r'Story by\s*:?', '', regex=True)
    .str.replace(r'Teleplay by\s*:?', '', regex=True)
    .str.replace(r'\n', ' ', regex=True))


In [37]:
# Separar autores correctamente
df3['written_by_clean'] = (df3['written_by_clean'].str.replace(r'\s*&\s*', ',', regex=True))

In [38]:
# Ahora arreglamos nombres pegados (ej: MalinsMarta)
df3['written_by_clean'] = df3['written_by_clean'].str.replace(r'([a-z])([A-Z])', r'\1, \2', regex=True)

In [39]:
df3['written_by_clean'].unique().tolist()

['David Crane,Marta Kauffman',
 'Jeffrey Astrof,Mike Sikowitz',
 'Alexa Junge',
 'Jeff Greenstein,Jeff Strauss',
 'Adam Chase,Ira Ungerleider',
 'Marta Kauffman,David Crane',
 'Jeffrey Astrof,Mike Sikowitz,Adam Chase,Ira Ungerleider',
 'Bill Lawrence',
 'Doty Abrams',
 'â\x80\x8a: David Crane,Marta Kauffmanâ\x80\x8a: Jeff Greenstein,Jeff Strauss',
 'Chris Brown',
 'Michael Curtis,Gregory S. Malins',
 'Betsy Borns',
 'Michael Borkow',
 'Ira Ungerleider',
 'Jeffrey Astrof,Mike Sikowitz, Michael Borkow',
 'Adam Chase',
 'Alexa Junge,Michael Borkow',
 'â\x80\x8a: Michael Curtis,Gregory S. Malinsâ\x80\x8a: Adam Chase',
 'Brian Buckner,Sebastian Jones',
 'Brown Mandell',
 'Ira Ungerleider,Brown Mandell',
 'Wil Calhoun',
 'Shana Goldberg-Meehan,Scott Silveri',
 'Seth Kurland',
 'Gregory S. Malins,Michael Curtis',
 'Ted Cohen,Andrew Reich',
 'â\x80\x8a: Pang-Ni Landrum,Mark Kunerthâ\x80\x8a: Shana Goldberg-Meehan,Scott Silveri',
 'â\x80\x8a: Pang-Ni Landrum,Mark Kunerthâ\x80\x8a: Adam Chase',


In [40]:
df3['written_by_clean'] = df3['written_by_clean'].str.replace(r'â\x80\x8a:?', '', regex=True)

In [41]:
# Arreglar casos donde hay coma dentro del nombre mal puesta:
df3['written_by_clean'] = df3['written_by_clean'].str.replace(r'(\b\w+),\s(\w+\b)', r'\1 \2', regex=True)

In [42]:
df3['written_by_clean'] = df3['written_by_clean'].str.replace(r'\s*&\s*', ',', regex=True)  # & → ,
df3['written_by_clean'] = df3['written_by_clean'].str.replace(r'\s*,\s*', ',', regex=True)  # quitar espacios alrededor de ,
df3['written_by_clean'] = df3['written_by_clean'].str.replace(r'([a-z])([A-Z])', r'\1, \2', regex=True)  # pegar nombre sin coma

In [43]:
df3['written_by_clean'].unique().tolist()

['David Crane,Marta Kauffman',
 'Jeffrey Astrof,Mike Sikowitz',
 'Alexa Junge',
 'Jeff Greenstein,Jeff Strauss',
 'Adam Chase,Ira Ungerleider',
 'Marta Kauffman,David Crane',
 'Jeffrey Astrof,Mike Sikowitz,Adam Chase,Ira Ungerleider',
 'Bill Lawrence',
 'Doty Abrams',
 ' David Crane,Marta Kauffman Jeff Greenstein,Jeff Strauss',
 'Chris Brown',
 'Michael Curtis,Gregory S. Malins',
 'Betsy Borns',
 'Michael Borkow',
 'Ira Ungerleider',
 'Jeffrey Astrof,Mike Sikowitz Michael Borkow',
 'Adam Chase',
 'Alexa Junge,Michael Borkow',
 ' Michael Curtis,Gregory S. Malins Adam Chase',
 'Brian Buckner,Sebastian Jones',
 'Brown Mandell',
 'Ira Ungerleider,Brown Mandell',
 'Wil Calhoun',
 'Shana Goldberg-Meehan,Scott Silveri',
 'Seth Kurland',
 'Gregory S. Malins,Michael Curtis',
 'Ted Cohen,Andrew Reich',
 ' Pang-Ni Landrum,Mark Kunerth Shana Goldberg-Meehan,Scott Silveri',
 ' Pang-Ni Landrum,Mark Kunerth Adam Chase',
 'Jill Condon,Amy Toomin',
 'Scott Silveri',
 'Shana Goldberg-Meehan',
 ' Ted Coh

In [44]:
correct_map_WR = { 
 'David Crane,Marta Kauffman':'David Crane,Marta Kauffman',
 'Jeffrey Astrof,Mike Sikowitz':'Jeffrey Astrof,Mike Sikowitz',
 'Alexa Junge':'Alexa Junge',
 'Jeff Greenstein,Jeff Strauss':'Jeff Greenstein,Jeff Strauss',
 'Adam Chase,Ira Ungerleider':'Adam Chase,Ira Ungerleider',
 'Marta Kauffman,David Crane':'Marta Kauffman,David Crane',
 'Jeffrey Astrof,Mike Sikowitz,Adam Chase,Ira Ungerleider':'Jeffrey Astrof,Mike Sikowitz,Adam Chase,Ira Ungerleider',
 'Bill Lawrence':'Bill Lawrence',
 'Doty Abrams':'Doty Abrams',
 ' David Crane,Marta Kauffman Jeff Greenstein,Jeff Strauss':'David Crane,Marta Kauffman,Jeff Greenstein,Jeff Strauss',
 'Chris Brown':'Chris Brown',
 'Michael Curtis,Gregory S. Malins':'Michael Curtis,Gregory S. Malins',
 'Betsy Borns':'Betsy Borns',
 'Michael Borkow':'Michael Borkow',
 'Ira Ungerleider':'Ira Ungerleider',
 'Jeffrey Astrof,Mike Sikowitz Michael Borkow':'Jeffrey Astrof,Mike Sikowitz,Michael Borkow',
 'Adam Chase':'Adam Chase',
 ' Alexa Junge,Michael Borkow':'Alexa Junge,Michael Borkow',
 ' Michael Curtis,Gregory S. Malins Adam Chase':'Michael Curtis,Gregory S. Malins,Adam Chase',
 'Brian Buckner,Sebastian Jones':'Brian Buckner,Sebastian Jones',
 'Brown Mandell':'Brown Mandell',
 'Ira Ungerleider Brown Mandell':'Ira Ungerleider,Brown Mandell',
 'Wil Calhoun':'Wil Calhoun',
 'Shana Goldberg-Meehan,Scott Silveri':'Shana Goldberg-Meehan,Scott Silveri',
 'Seth Kurland':'Seth Kurland',
 'Gregory S. Malins,Michael Curtis':'Gregory S. Malins,Michael Curtis',
 'Ted Cohen,Andrew Reich':'Ted Cohen,Andrew Reich',
 ' Pang-Ni Landrum,Mark Kunerth Shana Goldberg-Meehan,Scott Silveri':'Pang-Ni Landrum,Mark Kunerth,Shana Goldberg-Meehan,Scott Silveri',
 ' Pang-Ni Landrum,Mark Kunerth Adam Chase':'Pang-Ni Landrum,Mark Kunerth,Adam Chase',
 'Jill Condon,Amy Toomin':'Jill Condon,Amy Toomin',
 'Scott Silveri':'Scott Silveri',
 'Shana Goldberg-Meehan':'Shana Goldberg-Meehan',
 ' Ted Cohen,Andrew Reich Wil Calhoun':'Ted Cohen,Andrew Reich,Wil Calhoun',
 ' Alicia Sky Varinaitis Scott Silveri,Shana Goldberg-Meehan':'Alicia Sky Varinaitis,Scott Silveri,Shana Goldberg-Meehan',
 ' Mark Kunerth Richard Goodman':' Mark Kunerth,Richard Goodman',
 ' Andrew Reich,Ted Cohen Jill Condon,Amy Toomin':'Andrew Reich,Ted Cohen,Jill Condon,Amy Toomin',
 'Scott Silveri,Wil Calhoun':'Scott Silveri,Wil Calhoun',
 ' Adam Chase Gregory S. Malins,Michael Curtis':'Adam Chase,Gregory S. Malins,Michael Curtis',
 ' Seth Kurland Gregory S. Malins,Michael Curtis':'Seth Kurland,Gregory S. Malins,Michael Curtis',
 'Michael Borkow Jill Condon,Amy Toomin Shana Goldberg-Meehan,Scott Silveri':'Michael Borkow,Jill Condon,Amy Toomin,Shana Goldberg-Meehan,Scott Silveri',
 'Michael Curtis':'Michael Curtis',
 'Gigi Mc Creery,Perry Rein':'Gigi Mc Creery,Perry Rein',
 'Gregory S. Malins':'Gregory S. Malins',
 ' Brian Boyle Suzie Villandry':'Brian Boyle,Suzie Villandry',
 'Alicia Sky Varinaitis':'Alicia Sky Varinaitis',
 ' Michael Curtis Seth Kurland':'Michael Curtis,Seth Kurland',
 ' Alicia Sky Varinaitis Gigi Mc Creery,Perry Rein':'Alicia Sky Varinaitis,Gigi Mc Creery,Perry Rein',
 'Andrew Reich,Ted Cohen':'Andrew Reich,Ted Cohen',
 'Shana Goldberg-Meehan,Seth Kurland':'Shana Goldberg-Meehan,Seth Kurland',
 ' Scott Silveri Gregory S. Malins':'Scott Silveri,Gregory S. Malins',
 ' Shana Goldberg-Meehan Wil Calhoun':'Shana Goldberg-Meehan,Wil Calhoun',
 'Ted Cohen,Andrew Reich Gregory S. Malins,Scott Silveri':'Ted Cohen,Andrew Reich,Gregory S. Malins,Scott Silveri',
 'Perry Rein,Gigi Mc Creery':'Perry Rein,Gigi Mc Creery',
 'Sherry Bilsing-Graham,Ellen Plummer':'Sherry Bilsing-Graham,Ellen Plummer',
 ' Andrew Reich,Ted Cohen Perry Rein,Gigi Mc Creery':'Andrew Reich,Ted Cohen,Perry Rein,Gigi Mc Creery',
 'Brian Boyle':'Brian Boyle',
 ' Zachary Rosenblatt Brian Boyle':'Zachary Rosenblatt,Brian Boyle',
 ' Shana Goldberg-Meehan Andrew Reich,Ted Cohen':'Shana Goldberg-Meehan,Andrew Reich,Ted Cohen',
 ' Seth Kurland Sherry Bilsing-Graham,Ellen Plummer':'Seth Kurland,Sherry Bilsing-Graham,Ellen Plummer',
 'Gregory S. Malins,Adam Chase David Crane,Marta Kauffman':'Gregory S. Malins,Adam Chase,David Crane,Marta Kauffman',
 ' Zachary Rosenblatt Adam Chase':'Zachary Rosenblatt,Adam Chase',
 ' Seth Kurland Gigi Mc Creery,Perry Rein':'Seth Kurland,Gigi Mc Creery,Perry Rein',
 ' David J. Lagana Scott Silveri':' David J. Lagana,Scott Silveri',
 ' Brian Caldirola Sherry Bilsing-Graham,Ellen Plummer':'Brian Caldirola,Sherry Bilsing-Graham,Ellen Plummer',
 'Shana Goldberg-Meehan,Scott Silveri Andrew Reich,Ted Cohen':'Shana Goldberg-Meehan,Scott Silveri,Andrew Reich,Ted Cohen',
 ' Wil Calhoun David Crane,Marta Kauffman':'Wil Calhoun,David Crane,Marta Kauffman',
 'Sherry Bilsing,Ellen Plummer':'Sherry Bilsing,Ellen Plummer',
 ' Earl Davis Patty Lin':'Earl Davis,Patty Lin',
 'Patty Lin':'Patty Lin',
 'Zachary Rosenblatt':'Zachary Rosenblatt',
 ' Ellen Plummer,Sherry Bilsing Brian Buckner,Sebastian Jones':'Ellen Plummer,Sherry Bilsing,Brian Buckner,Sebastian Jones',
 ' Vanessa Mc Carthy Ellen Plummer,Sherry Bilsing':'Vanessa Mc Carthy,Ellen Plummer,Sherry Bilsing',
 ' Sherry Bilsing,Ellen Plummer Andrew Reich,Ted Cohen':'Sherry Bilsing,Ellen Plummer,Andrew Reich,Ted Cohen',
 ' Brian Buckner,Sebastian Jones Zachary Rosenblatt':'Brian Buckner,Sebastian Jones,Zachary Rosenblatt',
 ' Brian Buckner,Sebastian Jones Andrew Reich,Ted Cohen':'Brian Buckner,Sebastian Jones,Andrew Reich,Ted Cohen',
 ' Sherry Bilsing,Ellen Plummer Brian Boyle':'Sherry Bilsing,Ellen Plummer,Brian Boyle',
 'Scott Silveri,Shana Goldberg-Meehan':'Scott Silveri,Shana Goldberg-Meehan',
 ' Gregory S. Malins Brian Buckner,Sebastian Jones':'Gregory S. Malins,Brian Buckner,Sebastian Jones',
 'Gregory S. Malins Marta Kauffman,David Crane':'Gregory S. Malins,Marta Kauffman,David Crane',
 'Dana Klein Borkow':'Dana Klein Borkow',
 'Mark Kunerth':'Mark Kunerth',
 'R. Lee Fleming Jr.':'R. Lee Fleming Jr.',
 ' Robert Carlock Brian Buckner,Sebastian Jones':'Robert Carlock,Brian Buckner,Sebastian Jones',
 'Robert Carlock':'Robert Carlock',
 'Vanessa Mc Carthy':'Vanessa Mc Carthy',
 ' R. Lee Fleming Jr. Steven Rosenhaus':'R. Lee Fleming Jr.,Steven Rosenhaus',
 ' Peter Tibbals Mark Kunerth':'Peter Tibbals,Mark Kunerth',
 ' Dana Klein Borkow Brian Buckner,Sebastian Jones':'Dana Klein Borkow,Brian Buckner,Sebastian Jones',
 'Scott Silveri Marta Kauffman,David Crane':'Scott Silveri,Marta Kauffman,David Crane',
 ' Judd Rubin Peter Tibbals':' Judd Rubin Peter Tibbals',
 ' Sherry Bilsing-Graham,Ellen Plummer Steven Rosenhaus':'Sherry Bilsing-Graham,Ellen Plummer,Steven Rosenhaus',
 'Peter Tibbals':'Peter Tibbals',
 ' Brian Buckner,Sebastian Jones Sherry Bilsing-Graham,Ellen Plummer':'Brian Buckner,Sebastian Jones,Sherry Bilsing-Graham,Ellen Plummer',
 ' Dana Klein Borkow Mark Kunerth':'Dana Klein Borkow,Mark Kunerth',
 ' Scott Silveri Robert Carlock':'Scott Silveri,Robert Carlock',
 'Shana Goldberg-Meehan,Scott Silveri Marta Kauffman,David Crane':'Shana Goldberg-Meehan,Scott Silveri,Marta Kauffman,David Crane',
 'Brian Buckner':'Brian Buckner',
 'Sebastian Jones':'Sebastian Jones',
 'Robert Carlock,Dana Klein Borkow':'Robert Carlock,Dana Klein Borkow',
 ' Robert Carlock Tracy Reilly':'Robert Carlock,Tracy Reilly',
 ' Mark Kunerth David Crane,Marta Kauffman':'Mark Kunerth,David Crane,Marta Kauffman',
 'Story byâ: Alexa JungeTeleplay byâ: Michael Borkow': 'Alexa Junge,Michael Borkow',
 'Story byâ: Ira UngerleiderTeleplay byâ: Brown Mandell': 'Ira Ungerleider,Brown Mandell'}

In [45]:
df3['written_by_clean'] = df3['written_by_clean'].map(correct_map_WR)

In [46]:
df3['written_by_clean'].isna().sum()

np.int64(2)

In [47]:
df3[df3['written_by_clean'].isna()]

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating,written_by_clean
41,2,18,The One Where Dr. Ramoray Dies,Michael Lembeck,"Alexa Junge,Michael Borkow",1996-03-21,30.1,8.5,NaN
47,2,24,The One with Barry and Mindy's Wedding,Michael Lembeck,"Ira Ungerleider,Brown Mandell",1996-05-16,29.0,8.2,NaN


In [48]:
df3.loc[
    (df3['written_by_clean'].isna()) &
    (df3['written_by'] == 'Alexa Junge,Michael Borkow'),
    'written_by_clean'
] = 'Alexa Junge,Michael Borkow'

df3.loc[
    (df3['written_by_clean'].isna()) &
    (df3['written_by'] == 'Ira Ungerleider,Brown Mandell'),
    'written_by_clean'
] = 'Ira Ungerleider,Brown Mandell'

In [49]:
df3.head(5)

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating,written_by_clean
0,1,1,The Pilot,James Burrows,David Crane & Marta Kauffman,1994-09-22,21.5,8.3,"David Crane,Marta Kauffman"
1,1,2,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,1994-09-29,20.2,8.1,"David Crane,Marta Kauffman"
2,1,3,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-10-06,19.5,8.2,"Jeffrey Astrof,Mike Sikowitz"
3,1,4,The One with George Stephanopoulos,James Burrows,Alexa Junge,1994-10-13,19.7,8.1,Alexa Junge
4,1,5,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,1994-10-20,18.6,8.5,"Jeff Greenstein,Jeff Strauss"


In [50]:
# Convertir a lista de autores
df3['writers_list'] = (df3['written_by_clean'].str.split(',').apply(lambda x: [w.strip() for w in x if w.strip()]))

In [51]:
df3.head(5)

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating,written_by_clean,writers_list
0,1,1,The Pilot,James Burrows,David Crane & Marta Kauffman,1994-09-22,21.5,8.3,"David Crane,Marta Kauffman","[David Crane, Marta Kauffman]"
1,1,2,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,1994-09-29,20.2,8.1,"David Crane,Marta Kauffman","[David Crane, Marta Kauffman]"
2,1,3,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-10-06,19.5,8.2,"Jeffrey Astrof,Mike Sikowitz","[Jeffrey Astrof, Mike Sikowitz]"
3,1,4,The One with George Stephanopoulos,James Burrows,Alexa Junge,1994-10-13,19.7,8.1,Alexa Junge,[Alexa Junge]
4,1,5,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,1994-10-20,18.6,8.5,"Jeff Greenstein,Jeff Strauss","[Jeff Greenstein, Jeff Strauss]"


In [52]:
df3['writers_list'].astype(str).unique()

array(["['David Crane', 'Marta Kauffman']",
       "['Jeffrey Astrof', 'Mike Sikowitz']", "['Alexa Junge']",
       "['Jeff Greenstein', 'Jeff Strauss']",
       "['Adam Chase', 'Ira Ungerleider']",
       "['Marta Kauffman', 'David Crane']",
       "['Jeffrey Astrof', 'Mike Sikowitz', 'Adam Chase', 'Ira Ungerleider']",
       "['Bill Lawrence']", "['Doty Abrams']",
       "['David Crane', 'Marta Kauffman', 'Jeff Greenstein', 'Jeff Strauss']",
       "['Chris Brown']", "['Michael Curtis', 'Gregory S. Malins']",
       "['Betsy Borns']", "['Michael Borkow']", "['Ira Ungerleider']",
       "['Jeffrey Astrof', 'Mike Sikowitz', 'Michael Borkow']",
       "['Adam Chase']", "['Alexa Junge', 'Michael Borkow']",
       "['Michael Curtis', 'Gregory S. Malins', 'Adam Chase']",
       "['Brian Buckner', 'Sebastian Jones']", "['Brown Mandell']",
       "['Ira Ungerleider', 'Brown Mandell']", "['Wil Calhoun']",
       "['Shana Goldberg-Meehan', 'Scott Silveri']", "['Seth Kurland']",
       "['Grego

In [53]:
gender_map_WR = {
    "David Crane": "M",
    "Marta Kauffman": "F",
    "Jeffrey Astrof": "M",
    "Mike Sikowitz": "M",
    "Alexa Junge": "F",
    "Jeff Greenstein": "M",
    "Jeff Strauss": "M",
    "Adam Chase": "M",
    "Ira Ungerleider": "M",
    "Bill Lawrence": "M",
    "Doty Abrams": "F",
    "Chris Brown": "M",
    "Michael Curtis": "M",
    "Gregory S. Malins": "M",
    "Betsy Borns": "F",
    "Michael Borkow": "M",
    "Brian Buckner": "M",
    "Sebastian Jones": "M",
    "Brown Mandell": "M",
    "Wil Calhoun": "M",
    "Shana Goldberg-Meehan": "F",
    "Scott Silveri": "M",
    "Seth Kurland": "M",
    "Ted Cohen": "M",
    "Andrew Reich": "M",
    "Pang-Ni Landrum": "F",
    "Mark Kunerth": "M",
    "Jill Condon": "F",
    "Amy Toomin": "F",
    "Alicia Sky Varinaitis": "F",
    "Richard Goodman": "M",
    "Gigi Mc Creery": "F",
    "Perry Rein": "M",
    "Brian Boyle": "M",
    "Suzie Villandry": "F",
    "Sherry Bilsing-Graham": "F",
    "Ellen Plummer": "F",
    "Zachary Rosenblatt": "M",
    "David J. Lagana": "M",
    "Brian Caldirola": "M",
    "Earl Davis": "M",
    "Patty Lin": "F",
    "Vanessa Mc Carthy": "F",
    "Sherry Bilsing": "F",
    "Dana Klein Borkow": "F",
    "R. Lee Fleming Jr.": "M",
    "Robert Carlock": "M",
    "Steven Rosenhaus": "M",
    "Peter Tibbals": "M",
    "Judd Rubin Peter Tibbals": "M",
    "Tracy Reilly": "F"}

In [54]:
def count_gender(writers):
    count_f = sum(gender_map_WR.get(w.strip(), 'U') == 'F' for w in writers)
    count_m = sum(gender_map_WR.get(w.strip(), 'U') == 'M' for w in writers)
    return pd.Series([count_f, count_m])

df3[['writers_female_count', 'writers_male_count']] = df3['writers_list'].apply(count_gender)

In [55]:
df3.head(10)

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating,written_by_clean,writers_list,writers_female_count,writers_male_count
0,1,1,The Pilot,James Burrows,David Crane & Marta Kauffman,1994-09-22,21.5,8.3,"David Crane,Marta Kauffman","[David Crane, Marta Kauffman]",1,1
1,1,2,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,1994-09-29,20.2,8.1,"David Crane,Marta Kauffman","[David Crane, Marta Kauffman]",1,1
2,1,3,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-10-06,19.5,8.2,"Jeffrey Astrof,Mike Sikowitz","[Jeffrey Astrof, Mike Sikowitz]",0,2
3,1,4,The One with George Stephanopoulos,James Burrows,Alexa Junge,1994-10-13,19.7,8.1,Alexa Junge,[Alexa Junge],1,0
4,1,5,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,1994-10-20,18.6,8.5,"Jeff Greenstein,Jeff Strauss","[Jeff Greenstein, Jeff Strauss]",0,2
5,1,6,The One with the Butt,Arlene Sanford,Adam Chase & Ira Ungerleider,1994-10-27,18.2,8.1,"Adam Chase,Ira Ungerleider","[Adam Chase, Ira Ungerleider]",0,2
6,1,7,The One with the Blackout,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-11-03,23.5,9.0,"Jeffrey Astrof,Mike Sikowitz","[Jeffrey Astrof, Mike Sikowitz]",0,2
7,1,8,The One Where Nana Dies Twice,James Burrows,Marta Kauffman & David Crane,1994-11-10,21.1,8.1,"Marta Kauffman,David Crane","[Marta Kauffman, David Crane]",1,1
8,1,9,The One Where Underdog Gets Away,James Burrows,Jeff Greenstein & Jeff Strauss,1994-11-17,23.1,8.2,"Jeff Greenstein,Jeff Strauss","[Jeff Greenstein, Jeff Strauss]",0,2
9,1,10,The One with the Monkey,Peter Bonerz,Adam Chase & Ira Ungerleider,1994-12-15,19.9,8.1,"Adam Chase,Ira Ungerleider","[Adam Chase, Ira Ungerleider]",0,2


In [56]:
df3['writers_total'] = df3['writers_female_count'] + df3['writers_male_count']
df3['writers_female_ratio'] = df3['writers_female_count'] / df3['writers_total']

In [57]:
df3.head(3)

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating,written_by_clean,writers_list,writers_female_count,writers_male_count,writers_total,writers_female_ratio
0,1,1,The Pilot,James Burrows,David Crane & Marta Kauffman,1994-09-22,21.5,8.3,"David Crane,Marta Kauffman","[David Crane, Marta Kauffman]",1,1,2,0.5
1,1,2,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,1994-09-29,20.2,8.1,"David Crane,Marta Kauffman","[David Crane, Marta Kauffman]",1,1,2,0.5
2,1,3,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-10-06,19.5,8.2,"Jeffrey Astrof,Mike Sikowitz","[Jeffrey Astrof, Mike Sikowitz]",0,2,2,0.0


In [58]:
def gender_mix(row):
    if row['writers_female_count'] == 0:
        return 'All Male'
    elif row['writers_male_count'] == 0:
        return 'All Female'
    else:
        return 'Mixed'

df3['writers_gender_mix'] = df3.apply(gender_mix, axis=1)

In [64]:
df3.head(10)

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating,written_by_clean,writers_list,writers_female_count,writers_male_count,writers_total,writers_female_ratio,writers_gender_mix
0,1,1,The Pilot,James Burrows,David Crane & Marta Kauffman,1994-09-22,21.5,8.3,"David Crane,Marta Kauffman","[David Crane, Marta Kauffman]",1,1,2,0.5,Mixed
1,1,2,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,1994-09-29,20.2,8.1,"David Crane,Marta Kauffman","[David Crane, Marta Kauffman]",1,1,2,0.5,Mixed
2,1,3,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-10-06,19.5,8.2,"Jeffrey Astrof,Mike Sikowitz","[Jeffrey Astrof, Mike Sikowitz]",0,2,2,0.0,All Male
3,1,4,The One with George Stephanopoulos,James Burrows,Alexa Junge,1994-10-13,19.7,8.1,Alexa Junge,[Alexa Junge],1,0,1,1.0,All Female
4,1,5,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,1994-10-20,18.6,8.5,"Jeff Greenstein,Jeff Strauss","[Jeff Greenstein, Jeff Strauss]",0,2,2,0.0,All Male
5,1,6,The One with the Butt,Arlene Sanford,Adam Chase & Ira Ungerleider,1994-10-27,18.2,8.1,"Adam Chase,Ira Ungerleider","[Adam Chase, Ira Ungerleider]",0,2,2,0.0,All Male
6,1,7,The One with the Blackout,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-11-03,23.5,9.0,"Jeffrey Astrof,Mike Sikowitz","[Jeffrey Astrof, Mike Sikowitz]",0,2,2,0.0,All Male
7,1,8,The One Where Nana Dies Twice,James Burrows,Marta Kauffman & David Crane,1994-11-10,21.1,8.1,"Marta Kauffman,David Crane","[Marta Kauffman, David Crane]",1,1,2,0.5,Mixed
8,1,9,The One Where Underdog Gets Away,James Burrows,Jeff Greenstein & Jeff Strauss,1994-11-17,23.1,8.2,"Jeff Greenstein,Jeff Strauss","[Jeff Greenstein, Jeff Strauss]",0,2,2,0.0,All Male
9,1,10,The One with the Monkey,Peter Bonerz,Adam Chase & Ira Ungerleider,1994-12-15,19.9,8.1,"Adam Chase,Ira Ungerleider","[Adam Chase, Ira Ungerleider]",0,2,2,0.0,All Male


In [60]:
df3.to_csv('files/friends_info_gender.csv', index=False)

In [61]:
df3[df3['writers_gender_mix'] == 'All Male']

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating,written_by_clean,writers_list,writers_female_count,writers_male_count,writers_total,writers_female_ratio,writers_gender_mix
2,1,3,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-10-06,19.50,8.2,"Jeffrey Astrof,Mike Sikowitz","[Jeffrey Astrof, Mike Sikowitz]",0,2,2,0.0,All Male
4,1,5,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,1994-10-20,18.60,8.5,"Jeff Greenstein,Jeff Strauss","[Jeff Greenstein, Jeff Strauss]",0,2,2,0.0,All Male
5,1,6,The One with the Butt,Arlene Sanford,Adam Chase & Ira Ungerleider,1994-10-27,18.20,8.1,"Adam Chase,Ira Ungerleider","[Adam Chase, Ira Ungerleider]",0,2,2,0.0,All Male
6,1,7,The One with the Blackout,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-11-03,23.50,9.0,"Jeffrey Astrof,Mike Sikowitz","[Jeffrey Astrof, Mike Sikowitz]",0,2,2,0.0,All Male
8,1,9,The One Where Underdog Gets Away,James Burrows,Jeff Greenstein & Jeff Strauss,1994-11-17,23.10,8.2,"Jeff Greenstein,Jeff Strauss","[Jeff Greenstein, Jeff Strauss]",0,2,2,0.0,All Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,10,4,The One with the Cake,Gary Halvorson,Robert Carlock,2003-10-23,18.77,8.3,Robert Carlock,[Robert Carlock],0,1,1,0.0,All Male
223,10,6,The One with Ross's Grant,Ben Weiss,Sebastian Jones,2003-11-06,20.38,8.2,Sebastian Jones,[Sebastian Jones],0,1,1,0.0,All Male
224,10,7,The One with the Home Study,Kevin S. Bright,Mark Kunerth,2003-11-13,20.21,8.3,Mark Kunerth,[Mark Kunerth],0,1,1,0.0,All Male
226,10,9,The One with the Birth Mother,David Schwimmer,Scott Silveri,2004-01-08,25.49,8.6,Scott Silveri,[Scott Silveri],0,1,1,0.0,All Male


In [ ]:
df3['nombre_columna'].min()
df3['nombre_columna'].max()
